# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [21]:
using Manifolds
using Manopt
using LinearAlgebra
using Random
using Plots
using LaTeXStrings

In [28]:
include("../../../src/decompositions/signals/naive_low_rank_approximation.jl")
include("../../../src/decompositions/signals/curvature_corrected_low_rank_approximation.jl")
include("../../../src/decompositions/signals/exact_low_rank_approximation.jl")
include("../../../src/decompositions/signals/stochastic_curvature_corrected_low_rank_approximation.jl")

include("../../../src/functions/loss_functions/curvature_corrected_loss.jl")
include("../../../src/functions/loss_functions/exact_loss.jl")

exact_loss (generic function with 2 methods)

### Load data and construct manifold ###

In [3]:
# load data
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M)
n = 100  # 100


100

In [4]:
e = 1. * Matrix(I, 3, 3)
# compute basis
Θ = get_basis(M, e, DefaultOrthonormalBasis())
#  construct data
τ = 2.  # variance
σ = .05  # variance
Xₑ = Θ.data[4]
print(Xₑ)

Random.seed!(31)
predata = [exp(M, e, sqrt(τ) * randn(1)[1] * Xₑ) for i in 1:n]

data = [exp(M, predata[i], random_tangent(M, predata[i], Val(:Gaussian), σ)) for i in 1:n]; # ∈ P(3)^n


[0.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 0.0]

In [5]:
# Export slice image
num_export = 10
asymptote_export_SPD("results/artificial1D_orig.asy", data=data[1:min(num_export,n)], scale_axes=(2,2,2)); 

### Construct low rank approximation ###

In [5]:
q = mean(M, data)
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

In [29]:
stochastic_curvature_corrected_low_rank_approximation(M, q, data, 1; max_iter=500); 

step_size = 1.1710574157931244e-5
Initial  F(x): 1.07408003505 | 
# 1     change: 0.000000171 |  F(x): 1.07407953956 | 
# 2     change: 0.000000023 |  F(x): 1.07407957115 | 
# 3     change: 0.000000281 |  F(x): 1.07408004588 | 
# 4     change: 0.000000192 |  F(x): 1.07408023922 | 
# 5     change: 0.000000158 |  F(x): 1.07408024525 | 
# 6     change: 0.000000050 |  F(x): 1.07408049339 | 
# 7     change: 0.000000281 |  F(x): 1.07408474988 | 
# 8     change: 0.000000197 |  F(x): 1.07408436445 | 
# 9     change: 0.000000093 |  F(x): 1.07408518186 | 
# 10    change: 0.000000245 |  F(x): 1.07408917191 | 
# 11    change: 0.000000082 |  F(x): 1.07408877243 | 
# 12    change: 0.000000047 |  F(x): 1.07408820694 | 
# 13    change: 0.000000251 |  F(x): 1.07408983647 | 
# 14    change: 0.000000100 |  F(x): 1.07409045691 | 
# 15    change: 0.000000170 |  F(x): 1.07408936868 | 
# 16    change: 0.000000111 |  F(x): 1.07409038715 | 
# 17    change: 0.000000039 |  F(x): 1.07409035888 | 
# 18    change: 

In [8]:
exact_low_rank_approximation(M, q, data, 1; stepsize=1/1000, max_iter=20); 

Initial  F(x): 0.13166108791 | 
# 1     change: 0.016613118 |  F(x): 0.12180849684 | 
# 2     change: 0.014484485 |  F(x): 0.11313608335 | 
# 3     change: 0.012803013 |  F(x): 0.10549364037 | 
# 4     change: 0.011457455 |  F(x): 0.09875258756 | 
# 5     change: 0.010367370 |  F(x): 0.09280214014 | 
# 6     change: 0.009473915 |  F(x): 0.08754634479 | 
# 7     change: 0.008733588 |  F(x): 0.08290174960 | 
# 8     change: 0.008113896 |  F(x): 0.07879554298 | 
# 9     change: 0.007590326 |  F(x): 0.07516404344 | 
# 10    change: 0.007144187 |  F(x): 0.07195145484 | 
# 11    change: 0.006761083 |  F(x): 0.06910882577 | 
# 12    change: 0.006429805 |  F(x): 0.06659316826 | 
# 13    change: 0.006141529 |  F(x): 0.06436670350 | 
# 14    change: 0.005889235 |  F(x): 0.06239621085 | 
# 15    change: 0.005667282 |  F(x): 0.06065246239 | 
# 16    change: 0.005471088 |  F(x): 0.05910972975 | 
# 17    change: 0.005296898 |  F(x): 0.05774535280 | 
# 18    change: 0.005141605 |  F(x): 0.05653936189

In [9]:
max_iter = 50

nR_q = []
nU = []
ccR_q = []
ccU = []
eR_q = []
eU = []
for i in 1:d
    println("#$(i) | computing naive low-rank approximation")
    nRr_q, nUr = naive_low_rank_approximation(M, q, data, i)
    push!(nR_q, nRr_q)
    push!(nU, nUr)
    println("#$(i) | computing curvature corrected low-rank approximation")
    ccRr_q, ccUr = curvature_corrected_low_rank_approximation(M, q, data, i; max_iter=max_iter); 
    push!(ccR_q, ccRr_q)
    push!(ccU, ccUr)
    println("#$(i) | computing exact low-rank approximation")
    eRr_q, eUr = exact_low_rank_approximation(M, q, data, i; stepsize=1/1000, max_iter=max_iter); 
    push!(eR_q, eRr_q)
    push!(eU, eUr)
end

#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
step_size = 1.171031512818198e-5
Initial  F(x): 1.07408003505 | 
# 1     change: 0.094748028 |  F(x): 0.07434697905 | 
# 2     change: 0.024368819 |  F(x): 0.00816660357 | 
# 3     change: 0.006293040 |  F(x): 0.00374786483 | 
# 4     change: 0.001635823 |  F(x): 0.00344872329 | 
# 5     change: 0.000429633 |  F(x): 0.00342802781 | 
# 6     change: 0.000114610 |  F(x): 0.00342654872 | 
# 7     change: 0.000031256 |  F(x): 0.00342643807 | 
# 8     change: 0.000008773 |  F(x): 0.00342642928 | 
# 9     change: 0.000002553 |  F(x): 0.00342642853 | 
# 10    change: 0.000000792 |  F(x): 0.00342642846 | 
The algorithm performed a step with a change (7.92165759873453e-7) less than 1.0e-6.
#1 | computing exact low-rank approximation
Initial  F(x): 0.13166108791 | 
# 1     change: 0.016613118 |  F(x): 0.12180849684 | 
# 2     change: 0.014484485 |  F(x): 0.11313608335 | 
# 3     change: 0.012803

In [10]:
ref_distance = sum(distance.(Ref(M), Ref(q), data).^2)

naive_tangent_distances_r = zeros(d)
predicted_naive_distances_r= zeros(d)
true_naive_distances_r= zeros(d)

curvature_corrected_tangent_distances_r = zeros(d)
predicted_curvature_corrected_distances_r = zeros(d)
true_curvature_corrected_distances_r = zeros(d)

exact_tangent_distances_r = zeros(d)
exact_distances_r= zeros(d)

for rank in 1:d
    naive_log_q_data_r = Symmetric.([sum([nR_q[rank][i] * nU[rank][k,i] for i in 1:rank]) for k in 1:n])
    curvature_corrected_log_q_data_r = Symmetric.([sum([ccR_q[rank][i] * ccU[rank][k,i] for i in 1:rank]) for k in 1:n])
    exact_log_q_data_r = Symmetric.([sum([eR_q[rank][i] * eU[rank][k,i] for i in 1:rank]) for k in 1:n])
    
    # expoentiate back
    naive_data_r = exp.(Ref(M), Ref(q), naive_log_q_data_r)
    curvature_corrected_data_r = exp.(Ref(M), Ref(q), curvature_corrected_log_q_data_r)
    exact_data_r = exp.(Ref(M), Ref(q), exact_log_q_data_r)


    # compute relative tangent space error
    naive_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - naive_log_q_data_r).^2) / ref_distance
    curvature_corrected_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - curvature_corrected_log_q_data_r).^2) / ref_distance
    exact_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - exact_log_q_data_r).^2) / ref_distance


    # compute relative manifold error
    predicted_naive_distances_r[rank] = curvature_corrected_loss(M, q, data, naive_log_q_data_r)
    true_naive_distances_r[rank] = exact_loss(M, q, data, naive_log_q_data_r)
    predicted_curvature_corrected_distances_r[rank] = curvature_corrected_loss(M, q, data, curvature_corrected_log_q_data_r)
    true_curvature_corrected_distances_r[rank] = exact_loss(M, q, data, curvature_corrected_log_q_data_r)
    exact_distances_r[rank] = exact_loss(M, q, data, exact_log_q_data_r)
end

In [11]:
# We want plots with (1) the lower bound error, (2) the actually uncorrected manifold error and (3) the corrected manifold error
plot(1:d, [naive_tangent_distances_r, true_naive_distances_r, true_curvature_corrected_distances_r, exact_distances_r], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(0,1), xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/artificial1D_errors_by_rank.png")
plot(1:d, [naive_tangent_distances_r .+ 1e-16, true_naive_distances_r .+ 1e-16, true_curvature_corrected_distances_r .+ 1e-16, exact_distances_r .+ 1e-16], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(1e-16,1), xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logerrors_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_logerrors_by_rank.png"

In [12]:
# It would be nice to also have a plot that tells us something about the error in predicting the manifold loss (using CCL) and the actual loss 
# (1) for the naive approach (2) for the curvature corrected approach
plot(1:d, [(predicted_naive_distances_r .+ 1e-16) ./ (true_naive_distances_r .+ 1e-16), (predicted_curvature_corrected_distances_r .+ 1e-16) ./ (true_curvature_corrected_distances_r .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
savefig("results/artificial1D_discrepancy_by_rank.png")
plot(1:d, [(predicted_naive_distances_r .+ 1e-16) ./ (true_naive_distances_r .+ 1e-16), (predicted_curvature_corrected_distances_r .+ 1e-16) ./ (true_curvature_corrected_distances_r .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logdiscrepancy_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_logdiscrepancy_by_rank.png"